In [1]:
# Build using the code @https://github.com/mfaruqui/retrofitting

In [9]:
import math
import scipy

In [10]:
from copy import deepcopy

In [11]:
from sentence_transformers import SentenceTransformer

In [14]:
embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

I0420 15:22:51.468164 4784264640 SentenceTransformer.py:29] Load pretrained SentenceTransformer: roberta-base-nli-stsb-mean-tokens
I0420 15:22:51.469924 4784264640 SentenceTransformer.py:32] Did not find a '/' or '\' in the name. Assume to download model from server.
I0420 15:22:51.471421 4784264640 SentenceTransformer.py:68] Load SentenceTransformer from folder: /Users/markberger/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-base-nli-stsb-mean-tokens.zip
I0420 15:22:51.473448 4784264640 configuration_utils.py:182] loading configuration file /Users/markberger/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-base-nli-stsb-mean-tokens.zip/0_RoBERTa/config.json
I0420 15:22:51.474535 4784264640 configuration_utils.py:199] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,


### After looking through the paper / code multiple times I can't really figure out how does the presented loss function with the Euclidian distance compare to the code below, as there is no optimization happening here? 

### The question is, how do I modify this code to look at minimizing the cosine similarity between the neighbour embeddings instead of euclidian

In [37]:
''' Retrofit word vectors to a lexicon '''
def retrofit(wordVecs, lexicon, numIters):
    newWordVecs = deepcopy(wordVecs)
    wvVocab = set(newWordVecs.keys())
    loopVocab = wvVocab.intersection(set(lexicon.keys()))
    for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
        for word in loopVocab:
            wordNeighbours = set(lexicon[word]).intersection(wvVocab)
            numNeighbours = len(wordNeighbours)
            #no neighbours, pass - use data estimate
            if numNeighbours == 0:
                continue
            # the weight of the data estimate is the number of neighbours
            newVec = numNeighbours * wordVecs[word]
            # loop over neighbours and add to new vector (currently with weight 1)
            for ppWord in wordNeighbours:
                newVec += newWordVecs[ppWord]
            newWordVecs[word] = newVec/(2*numNeighbours)
    return newWordVecs


In [38]:
# Introduce also the degree of the paper (essentially the amount of references the paper has) --> thats the number of neighbours?

In [39]:
corpus_embeddings = embedder.encode(["Hello my name is Mark", "I come from a land far away.", "Hi, here we go!", 'My cat is called Dimitri'])

Batches: 100%|██████████| 1/1 [00:00<00:00,  8.98it/s]


In [40]:
sentence_embs = {}
sentence_embs["name"] = corpus_embeddings[0] / math.sqrt((corpus_embeddings[0]**2).sum() + 1e-6)
sentence_embs["land"] = corpus_embeddings[1] / math.sqrt((corpus_embeddings[1]**2).sum() + 1e-6)
sentence_embs["hi"] = corpus_embeddings[2] / math.sqrt((corpus_embeddings[2]**2).sum() + 1e-6)
sentence_embs["cat"] = corpus_embeddings[3] / math.sqrt((corpus_embeddings[3]**2).sum() + 1e-6)

In [41]:
lexicon_papers = {}
lexicon_papers["name"] = ["hi", "land"]

In [46]:
retro_name = retrofit(sentence_embs, lexicon_papers, 10)["name"]
retro_name_less = retrofit(sentence_embs, lexicon_papers, 1)["name"]

In [47]:
retro_name == retro_name_less

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [34]:
distances = scipy.spatial.distance.cdist([retro_name_less], [sentence_embs["name"]], "cosine")[0]

In [35]:
1 - distances

array([0.89655808])

In [22]:
sentence_embs["name"] /= math.sqrt((sentence_embs["name"]**2).sum() + 1e-6)

In [23]:
math.sqrt((sentence_embs["name"]**2).sum() + 1e-6)

0.9999999635581964

In [24]:
corpus_embeddings[0][500] / math.sqrt((sentence_embs["name"]**2).sum() + 1e-6)

0.5246709776389387